In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing import sequence
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
import numpy as np

In [ ]:
max_caption_len = 3
vocab_size = 3

In [ ]:
images = []
for i in range(2):
    images.append(img_to_array(load_img('/data/raw/screenshot.jpg', target_size=(224, 224))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

In [ ]:
html_input = np.array(
            [[[0., 0., 0.], #start
             [0., 0., 0.],
             [1., 0., 0.]],
             [[0., 0., 0.], #start <HTML>Hello World!</HTML>
             [1., 0., 0.],
             [0., 1., 0.]]])

next_words = np.array(
            [[0., 1., 0.], # <HTML>Hello World!</HTML>
             [0., 0., 1.]]) # end

In [ ]:
VGG = VGG16(weights='None', include_top=True)
VGG.load_weights('/data/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5')
VGG.trainable = False
vgg_out = RepeatVector(max_caption_len)(VGG.output)

In [ ]:
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(5, return_sequences=True)(language_input)

In [ ]:
decoder = concatenate([vgg_out, language_model])
decoder = LSTM(5, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)
model = Model(inputs=[VGG.input, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
model.fit([images, html_input], next_words, batch_size=2, shuffle=False, epochs=1000)

In [ ]:
start_token = [1., 0., 0.]
sentence = np.zeros((1, 3, 3))
sentence[0][2] = start_token

In [ ]:
first_word = model.predict([np.array([images[1]]), sentence])

In [ ]:
sentence[0][1] = start_token
sentence[0][2] = np.round(first_word)

In [ ]:
second_word = model.predict([np.array([images[1]]), sentence])

In [ ]:
sentence[0][0] = start_token
sentence[0][1] = np.round(first_word)
sentence[0][2] = np.round(second_word)

In [ ]:
print(sentence)